In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import datetime as dt
from selenium.common.exceptions import StaleElementReferenceException, WebDriverException
import re
from bs4 import BeautifulSoup
from time import sleep

In [2]:
url = 'https://line4bet.ru/1x-13-03-2023-football/'

In [3]:
driver = webdriver.Chrome()
driver.get(url)
assert 'Движение линий' in driver.title

In [4]:
# "Теннис" button searching
tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
assert len(tennis_btn) > 0
tennis_btn[0].click()

In [5]:
# looking for available date
previous_date = dt.datetime.today() - dt.timedelta(days=1)
prev_day = previous_date.day
dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
try:
    dates_objects[prev_day-1].click()
except:
    dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
    dates_objects[prev_day-1].click()

In [6]:
def get_data_event(event):
    cleaned_event = re.sub('<tbody>|<tr>|<td>|</tbody>|</tr>|</td>|<td colspan="2">|\t','', event)
    nsymbols_data = 16
    date = cleaned_event[:nsymbols_data]
    cleaned_event_ = cleaned_event[nsymbols_data:].strip()
    splitted_event = cleaned_event_.split(';')
    players = splitted_event[0]
    cleaned_event_ = splitted_event[1].strip()
    if 'Победа' in cleaned_event:
        winner = cleaned_event_.split(', ')[1].split(',')[0].strip()
        splitted_event = cleaned_event_.split(', ')[0].split('  ')
        event_ = ' '.join(splitted_event[1:3])
        score = splitted_event[-1]
    else:
        winner = ''
        event_ = ' '.join(cleaned_event_.split('  ')[1:3])
        score = ''
    return date, event_, players, winner, score


def get_data_table(tables):
    page_tables = []
    for str_table in tables:
        soup = BeautifulSoup(str_table, 'html.parser')
        cleaned_table_data = [
            re.sub('<tr> | </tr>| class="c1"| class="c2"', '', str(x)) for x in soup.find_all('tr')[1:-1]
        ]
        columns = soup.find_all('th')
        columns = [re.sub('<th> | </th>', '', str(x)) for x in columns]
        table = []
        for str_ in cleaned_table_data: 
            soup_str = BeautifulSoup(str_, 'html.parser')
            table.append([x.text for x in soup_str.find_all('td')])
        table = pd.DataFrame(data=table, columns=columns)
        page_tables.append(table)
    return page_tables


def restart_page(driver):
    tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
    tennis_btn[0].click()
    # looking for available date
    previous_date = dt.datetime.today() - dt.timedelta(days=1)
    prev_day = previous_date.day
    dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
    try:
        dates_objects[prev_day-1].click()
    except:
        dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
        dates_objects[prev_day-1].click()

In [7]:
bcs = ['tab_1x', 'tab_bc', 'tab_fb', 'tab_mf']
data_events = {bc: [] for bc in bcs}
data_tables = {bc: [] for bc in bcs}
for bc in bcs:
    sleep(1)
    driver.find_elements(By.ID, bc)[0].click()
    sleep(1)
    npages = driver.find_elements(By.CLASS_NAME, 'pages')
    # pages iterating
    for i in range(len(npages)):
        try:
            driver.execute_script("arguments[0].click();", npages[i])
        except (StaleElementReferenceException, WebDriverException):
            npages = driver.find_elements(By.CLASS_NAME, 'pages')
            driver.execute_script("arguments[0].click();", npages[i])    
        sleep(1)
        html_source_code = driver.execute_script("return document.body.innerHTML;")
        html_soup = BeautifulSoup(html_source_code, 'html.parser')
        tbody_data = np.array([str(x) for x in html_soup.find_all(["tbody"])], dtype='object')
        # events parsing
        events_indexes = ['<tbody><tr><td>' in x and 'colspan="2"' in x for x in tbody_data]
        events = tbody_data[events_indexes]
        # data_events.append([get_data_event(x) for x in events])
        data_events[bc].append([get_data_event(x) for x in events])
        # tables parsing
        tables = tbody_data[['Дата-время скана линии' in x for x in tbody_data]]
        # data_tables.append(get_data_table(tables))
        data_tables[bc].append(get_data_table(tables))

In [40]:
np.vstack(data_events['tab_1x']).shape

(147, 5)

In [56]:
np.hstack(np.array(data_tables['tab_1x'], dtype='object')).size

C:\Users\dimaz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


147

In [57]:
50 + 55 + 42

147

correct -> 50, 55, 42